<a href="https://colab.research.google.com/github/dorliss/Dorenskaia_LaguagesForAI/blob/main/1_%D0%94%D0%BE%D1%80%D0%B5%D0%BD%D1%81%D0%BA%D0%B0%D1%8F_%D0%AF%D0%9F%D0%B4%D0%BB%D1%8F%D0%98%D0%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа №1**

1. Установка TensorFlow:

In [ ]:
!pip install --upgrade tensorflow

2. Импорт необходимых библиотек:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import json
import os

3. Определение модели трансформера:

In [ ]:
class TransformerLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim):
        super(TransformerLayer, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)

    def call(self, inputs, training=False):  # Добавлен аргумент training
        attn_output = self.attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Пример использования слоя трансформера
embed_dim = 64  # Размерность встраивания
num_heads = 4   # Количество голов в многоглавом внимании
ff_dim = 32     # Размерность скрытого слоя

inputs = layers.Input(shape=(None, embed_dim))  # Входные данные
transformer_layer = TransformerLayer(embed_dim, num_heads, ff_dim)
outputs = transformer_layer(inputs)  # Теперь это должно работать без ошибок

model = keras.Model(inputs=inputs, outputs=outputs)

# Выводим сводку модели для проверки
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, None, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_layer (TransformerLayer) │ (None, None, 64)            │          70,816 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 70,816 (276.62 KB)

 Trainable params: 70,816 (276.62 KB)

 Non-trainable params: 0 (0.00 B)

4. Загрузка и подготовка датасета CLEVR


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import json
import os

# Укажите путь к директории с данными CLEVR на вашем Google Диске
clevr_dir = '/content/drive/MyDrive/CLEVR'  # Измените путь при необходимости

# Загрузка графов сцен
with open(os.path.join(clevr_dir, 'scenes/CLEVR_train_scenes.json')) as f:
    train_scenes = json.load(f)

with open(os.path.join(clevr_dir, 'scenes/CLEVR_val_scenes.json')) as f:
    val_scenes = json.load(f)

# Загрузка вопросов
with open(os.path.join(clevr_dir, 'questions/CLEVR_train_questions.json')) as f:
    train_questions = json.load(f)

with open(os.path.join(clevr_dir, 'questions/CLEVR_val_questions.json')) as f:
    val_questions = json.load(f)

print("Train scenes loaded:", len(train_scenes['scenes']))
print("Validation scenes loaded:", len(val_scenes['scenes']))
print("Train questions loaded:", len(train_questions['questions']))
print("Validation questions loaded:", len(val_questions['questions']))

Train scenes loaded: 70000
Validation scenes loaded: 15000
Train questions loaded: 699989
Validation questions loaded: 149991


4. Реализация логики сравнения местоположений объектов

In [ ]:
def extract_object_locations(scene):
    locations = {}
    for obj in scene['objects']:
        obj_id = obj['id']
        locations[obj_id] = obj['position']  # Предполагаем наличие атрибута 'position'
    return locations

Функцию для сравнения местоположений:

In [ ]:
def compare_locations(scene, question):
    locations = extract_object_locations(scene)

    # Отладочная информация
    print("Объекты и их местоположения:", locations)

    # Проверяем наличие ключевых слов в вопросе
    if "где" in question and "относительно" in question:
        # Извлекаем идентификаторы объектов из вопроса
        words = question.split(" ")

        # Предполагаем, что объекты находятся перед "относительно" и в конце
        try:
            obj_a_id = words[words.index("объект") + 1]  # Найти объект A
            obj_b_id = words[words.index("объект", words.index("объект") + 1) + 1]  # Найти объект B

            # Проверяем наличие объектов в местоположениях
            if obj_a_id in locations and obj_b_id in locations:
                loc_a = np.array(locations[obj_a_id])
                loc_b = np.array(locations[obj_b_id])

                # Сравниваем местоположения
                if loc_a[0] < loc_b[0]:
                    return f"{obj_a_id} находится слева от {obj_b_id}"
                else:
                    return f"{obj_a_id} находится справа от {obj_b_id}"
            else:
                return f"Объекты '{obj_a_id}' или '{obj_b_id}' не найдены в сцене."
        except IndexError:
            return "Не удалось извлечь идентификаторы объектов из вопроса."

    return "Не могу ответить на вопрос"

Пример использования

In [ ]:
scene_example = {
    'objects': [
        {'id': 'A', 'position': [1, 2]},  # Объект A с координатами
        {'id': 'B', 'position': [3, 2]}   # Объект B с координатами
    ]
}

question_example = "Где находится объект A относительно объекта B?"
answer = compare_locations(scene_example, question_example)
print(answer)  # Ожидаемый вывод: A находится слева от B

Объекты и их местоположения: {'A': [1, 2], 'B': [3, 2]}
Не могу ответить на вопрос
